In [1]:
import numpy as np

# Calculate the energy of matrix A
def energy(A):
    total_energy = 0
    for i in range(A.shape[1]):
        if np.sum(A[:, i]) != 0:
            total_energy += np.dot(A[:, i].conj(), A[:, i])
    return total_energy


# Sampling function with weights and mapping
def sampling(M, nh, _, nx, map_, weights):
    d = np.zeros(nh, dtype=complex)
    M = M.flatten() * weights.flatten()
    m = np.fft.ifft(M)[:nx]
    checkshape(d, m, weights)
    for i in range(nh):
        d[i] = m[map_[i]]
    return d


# Interpolate function with weights and mapping
def interpolate(d, nh, nk, nx, map_, weights):
    m = np.zeros(nx, dtype=complex)
    checkshape(d, m, weights)
    for i in range(nh):
        m[map_[i]] = d[i]
    M = np.fft.fft(m, nk) / nk
    return M * weights.flatten()


# Shape check function
def checkshape(d, x, w):
    if d.ndim > 1 and d.shape[1] > 1:
        raise ValueError("d is not a vector")
    if x.ndim > 1 and x.shape[1] > 1:
        raise ValueError("x is not a vector")
    if w.ndim > 1 and w.shape[1] > 1:
        raise ValueError("w is not a vector")


# Upsample data based on mapping
def upsample(d, map_, nx):
    nt, nh = d.shape
    m = np.zeros((nt, nx), dtype=d.dtype)
    for i in range(nh):
        if map_[i] > 0:
            m[:, map_[i]] = d[:, i]
    return m


# Downsample data based on mapping
def downsample(m, map_, nh):
    nt, _ = m.shape
    d = np.zeros((nt, nh), dtype=m.dtype)
    for i in range(nh):
        if map_[i] > 0:
            d[:, i] = m[:, map_[i]]
    return d


# Generate a mapping array for interpolation
def mapping(h, x):
    nx, nh = len(x), len(h)
    map_ = np.zeros(nh, dtype=int)
    for i in range(nh):
        for j in range(nx):
            if round(h[i]) == round(x[j]):
                map_[i] = j
    return map_


# Weighted Conjugate Gradient Least Squares (WTCGLS) for MWNI interpolation
def wtcgls_mwni(d, nh, nk, nx, map_, w, niter, _):
    eps = 1e-6
    X = np.zeros(nk, dtype=complex)
    y = np.concatenate([d.flatten(), X])
    y0 = sampling(X, nh, nk, nx, map_, w)
    y0 = np.concatenate([y0[:nh], eps * X[:nk]])
    rr = y - y0
    energy = np.dot(rr.conj().T, rr)
    gammam = np.dot(rr.conj().T, interpolate(rr, nh, nk, nx, map_, w) + eps * rr[nh:nh + nk])

    for iter in range(niter):
        ss = sampling(S, nh, nk, nx, map_, w)
        ss = np.concatenate([ss[:nh], eps * S[:nk]])
        alpha = gammam / np.dot(ss.conj().T, ss)
        X += alpha * S
        rr -= alpha * ss
        G = interpolate(rr, nh, nk, nx, map_, w) + eps * rr[nh:nh + nk]
        gammam_new = np.dot(G.conj().T, G)
        beta = gammam_new / gammam
        S = G + beta * S
        gammam = gammam_new

        rms = np.sqrt(np.dot(rr.conj().T, rr) / len(rr))
        if rms < 1e-2:
            break

    X *= w.flatten()
    x = np.fft.ifft(X)[:nx]
    w = smoothing(np.abs(np.fft.fft(x, nk)), 5)
    return x, w, iter + 1


# Apply band limitation to weights
def band_limit(w, nk):
    li, ui = round(nk / 4 + 1), round(3 * nk / 4)
    w[li:ui] = 0
    return w


# Apply smoothing with a window size
def smoothing(x, nl):
    y = x.copy()
    nl2 = (nl - 1) // 2
    sum_ = np.sum(x[:nl])
    for i in range(nl2 + 1, len(x) - nl2):
        sum_ += x[i + nl2] - x[i - nl2 - 1]
        y[i] = sum_ / nl
    return y


# Dot product test for validation
def dotproduct_test(d, m, w, nh, nk, nx, map_):
    D1 = np.random.random(d.shape) + 1j * np.random.random(d.shape)
    M1 = np.random.random(m.shape) + 1j * np.random.random(m.shape)
    D2 = sampling(M1, nh, nk, nx, map_, np.random.random(w.shape))
    D2 = np.concatenate([D2[:nh], eps * M1[:nk]])
    M2 = interpolate(D1, nh, nk, nx, map_, np.random.random(w.shape))
    M2 += eps * D1[nh:nh + nk]
    return np.abs(np.dot(D1.conj().T, D2) / np.dot(M1.conj().T, M2))


# Duplicate half of the FFT to create a full spectrum
def duplic(in_):
    NF, NP = in_.shape
    if NF < NP:
        in_ = in_.T
    out = np.zeros((2 * NF, NP), dtype=in_.dtype)
    out[:NF, :] = in_
    out[NF, :] = in_[NF - 1, :].real
    out[NF + 1:, :] = np.conj(np.flipud(in_[:NF - 1, :]))
    return out


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Configuration and initial setup
ninterp = 2
firstfig = 1
method = 'reg'
doplot = 2
method = method[:3]
testnumber = 2

# Load data based on test number
if testnumber == 1:
    data = np.loadtxt('data2.dat')  # replace with correct format or loader
elif testnumber == 2:
    data = np.loadtxt('curved_noisy.dat')
elif testnumber == 3:
    data = np.loadtxt('linear_clean_data.dat')
elif testnumber == 4:
    data = np.loadtxt('linear_noisy_data.dat')


In [ ]:

# Placeholder for adding noise
# for k in range(data.shape[1]):
#     data[:, k] += rnoise(data[:, k], 1)
#     datakill[:, k] += rnoise(datakill[:, k], 1)

# Initializations
nz = 10  # zero padding in Kx
nt = len(t)  # assuming `t` is defined or loaded previously
dt = t[1] - t[0]

# Generate regular h from irregular h
dh = (h[-1] - h[0]) / (len(h) - 1)
hi = h
h = np.arange(hi[0], hi[-1] + dh, dh)

if len(h) != len(hi):
    print("Error in h length")
    exit()

# Initialize data with gaps filled
dataorig = data.copy()
hh, data, killnumber = [], [], []

for i in range(datakill.shape[1]):
    if np.sum(datakill[:, i]) != 0:
        hh.append(h[i])
        data.append(datakill[:, i])
    else:
        killnumber.append(i)

h = np.array(hh)
data = np.array(data).T

# Create output axis and other configurations
nh = len(h)
dx = dh / ninterp
x = np.arange(h[0], h[-1] + dx, dx)
nx = len(x)
nk = nx + nz

# Placeholder for mapping and upsample functions
map_ = mapping(h, x)
model = upsample(data, map_, nx)

print("Size of model:", model.shape)

# Fourier transform and processing
D = np.fft.fft(data, nt, axis=0)
nf = nt // 2
df = 1 / (dt * nt)
w = 2 * np.pi * np.arange(nf) * df
maxfreqHz = 100
maxfreq = nf
freqaxis = np.arange(maxfreq) * df

DH = D[:nf, :]
MH = np.zeros((nf, nx), dtype=complex)
weights = np.ones(nk)

# Test dot product (requires implementation)
test = dotproduct_test(np.concatenate([DH[0, :], np.zeros(nk)]), np.fft.fft(MH[0, :], nk), weights, nh, nk, nx, map_)
print("Test =", test)

weights = band_limit(weights, nk)
imaxfreq = int(round(maxfreq))
Etotal = np.zeros(imaxfreq)

# Iterate over frequencies
for f in range(imaxfreq):
    E = energy(DH[f, :]) / (nh * nh)
    if E == 0:
        continue
    Etotal[f] = E
    DH[f, :] /= E
    MH[f, :], weights, NITER_f = wtcgls_mwni(DH[f, :], nh, nk, nx, map_, weights, 200, f)
    DH[f, :] *= E
    MH[f, :] *= E

    # Optional plotting per frequency
    if doplot == 1:
        plt.subplot(311)
        plt.plot(np.abs(np.fft.fft(DH[f, :])))
        plt.subplot(312)
        plt.plot(np.abs(np.fft.fft(MH[f, :])))
        plt.subplot(313)
        plt.plot(weights)
        plt.show()

# Padding and inverse FFT to obtain interpolated model
MH[imaxfreq:] = 0
M = duplic(MH)
model = np.real(np.fft.ifft(M, axis=0))[:nt, :]
modelreg = model.copy()

if ninterp > 1:
    model = model[:, ::ninterp]

print("Size of model:", model.shape)

# Interpolation
index1 = np.arange(len(hi))
index2 = np.linspace(0, len(hi) - 1, len(hi) * ninterp)
xi = np.interp(index2, index1, hi)

# Plotting results
if doplot > 1:
    plt.figure(firstfig)
    plt.subplot(221)
    wigb(data, 1, hh, t)
    plt.subplot(222)
    wigb(model, ninterp, hi, t)

    plt.figure(firstfig + 1)
    plt.imshow(np.fft.fftshift(np.abs(np.fft.fft2(data))), aspect='auto')
    plt.figure(firstfig + 2)
    plt.imshow(np.fft.fftshift(np.abs(np.fft.fft2(model))), aspect='auto')

    residual = model - dataorig
    plt.figure()
    plt.subplot(221)
    wigb(dataorig)
    plt.subplot(222)
    wigb(datakill)
    plt.subplot(223)
    wigb(model)
    plt.subplot(224)
    wigb(residual)
    plt.show()

# Optional save of model
if testnumber == 1:
    np.save("model.npy", model)
elif testnumber == 2:
    np.save("curved_noisy_model2.npy", model)
elif testnumber == 3:
    np.save("linear_clean_model.npy", model)
elif testnumber == 4:
    np.save("linear_noisy_model.npy", model)
